In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00


In [3]:
import cv2
import os
from ultralytics import YOLO
from IPython.display import Video, display


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
INPUT_VIDEO = "/content/drive/MyDrive/cctv.mp4/3603382381-preview.mp4"
OUTPUT_PATH = "/content/output_tracking_yolo11.mp4"
COMPRESSED_PATH = "/content/output_tracking_yolo11_compressed.mp4"

In [5]:
model = YOLO("yolo11n.pt")  # Latest YOLO-11 Nano

In [6]:
cap = cv2.VideoCapture(INPUT_VIDEO)

if not cap.isOpened():
    raise IOError("Error opening video file")

fps = int(cap.get(cv2.CAP_PROP_FPS))
TARGET_WIDTH, TARGET_HEIGHT = 640, 360

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (TARGET_WIDTH, TARGET_HEIGHT))

unique_person_ids = set()

print("Processing video with YOLO-11...")

Processing video with YOLO-11...


In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))

    results = model.track(
        frame,
        persist=True,
        classes=0,           # Person class
        tracker="bytetrack.yaml",
        verbose=False
    )

    result = results[0]

    if result.boxes.id is not None:
        boxes = result.boxes.xyxy.cpu().numpy()
        ids = result.boxes.id.cpu().numpy()

        for box, track_id in zip(boxes, ids):
            unique_person_ids.add(int(track_id))
            x1, y1, x2, y2 = map(int, box)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"ID: {int(track_id)}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2
            )

    cv2.putText(
        frame,
        f"Count: {len(unique_person_ids)}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2
    )

    out.write(frame)

cap.release()
out.release()

print("Processing complete!")

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 233ms
Prepared 1 package in 119ms
Installed 1 package in 1ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 1.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Processing complete!


In [8]:
if os.path.exists(OUTPUT_PATH):
    os.system(f"ffmpeg -y -i {OUTPUT_PATH} -vcodec libx264 {COMPRESSED_PATH}")
    display(Video(COMPRESSED_PATH, embed=True, width=640))
else:
    print("Error: Output file not found")